In [ ]:
import geopandas as gpd
from shapely import wkt
import os
from glob import glob
from src import (batch_wms_download, write_vrt, image_thresh,
                 raster2array, do_ac, ms_snake)
from tqdm import tqdm

Sort geometry....

In [ ]:
maindir = os.getcwd()
# The csv you shared with me
geomcsv = ('ErosionMapping.csv')

gdf  = gpd.read_file(geomcsv)
# apply as geometry just so we can use if required 
# btw just moving the column to geometry doesn't work - format must be not right
gdf['geometry'] = gdf.wkt_geom.apply(wkt.loads)
#dump
gdf = gdf.drop(columns='wkt_geom')

In [ ]:
# subset of interest
monad = gdf[gdf['Region'] == 'Monadhliath']

wms = ('http://www.getmapping.com/wms/scotland/'
       'scotland.wmsx?login=jhilogin02&password=j54apd')

layer = 'ScotlandBest250mm'

Download

In [ ]:
outdir = ('tiles')

os.mkdir(outdir)

batch_wms_download(monad, wms, layer, outdir, attribute='id',
                       espg='27700')

Make a virtual raster should you want to view in qgis as one

In [ ]:
vrt = os.path.join(maindir, os.path.join('monad.vrt'))

infiles = glob(os.path.join(outdir, '*.tif'))
infiles.sort()

write_vrt(infiles, vrt)

Make a list of output strings for later ac shapefiles & rasters

In [ ]:
acdir = os.path.join(maindir, 'actiles')
os.mkdir(acdir)
# haha ugly
ootlist = [os.path.join(acdir, os.path.split(i)[1])[:-4]+'_ac.shp' for
           i in infiles]

For each file I choose a threshold manualy with napari 

This doesn't return anything just to mess about with thresholds as it will differ per image. Tried algorithmic solutions but better off by eye in this case....

In [ ]:
img = raster2array(infiles[0])

image_thresh(img)

Having "established" what the img thesholds are for each image in the list, run the process. This will save the aforementioned shapefiles of the AC results.

The plot won't update in jupyter which is annoying so just look at it in qgis.


In [ ]:
threshes = [70, 55, 50, 55, 55, 55]

for f, i, t in tqdm(zip(infiles, ootlist, threshes)):
    
    do_ac(f, i, iterations=10, thresh=t,
          smoothing=1, lambda1=1, lambda2=1, area_thresh=4, vis=False)

Alternatively you could draw them roughly then use ac to alter them per polygon
 though care must be taken that the polygon or respective neigbourhood around it
 (ie bufs 1 & 2) doesn't fall of edge of image (see args in lib or src)
 If there are overlapping polygons beforehand best disolve 
 then single > multipart them 

In [ ]:
inshp = ('my/erosion.shp') # replace with your own path/file
outshp = ('my/acedit.shp') # replace with your own path/file

ms_snake(inshp, vrt, outshp,  band=2, buf1=0, buf2=0, 
           algo="ACWE", nodata_value=0,
           iterations=50,  smoothing=1, lambda1=1, lambda2=1,
           threshold='auto', balloon=-1)
